In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('fireCallApp').getOrCreate()

24/12/10 10:01:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 샌프란시스코 소방서 호출 데이터셋
1. CallNumber : 호출 번호
2. UnitID : 응답한 소방서 장비의 ID
3. IncidentNumber : 사건 번호
4. CallType : 호출 유형
5. CallFinalDisposition : 호출 처리 결과
6. Address
7. City 
8. Zipcode
9. Battalion : 소방 배타리온
10. StationArea : 해당 호출을 처리한 소방서 지역
11. Box : 사건이 발생한 특정 구역을 나타내는 코드
12. OriginalPriority : 사건 발생 당시 부여된 우선순위
13. Priority : 사건 처리의 긴급도나 중요도
14. FinalPriority : 사건이 처리된 후 결정된 우선순위
15. ALSUnit : 고급 생명 지원(Advanced Life Support) 유닛의 여부
16. CallTypeGroup 
17. NumAlarms : 발령된 경고 횟수
18. UnitType : 응답한 유닛의 유형
19. UnitSequenceInCallDispatch : 호출 배정 시 유닛의 순서
20. FirePreventionDistrict : 화재 예방 구역
21. SupervisorDistrict
22. Neighborhood 
23. Location
24. RowID : 각 행 고유 식별자
25. ResponseDelayedinMins : 응답 지연 시간
26. IncidentDate
27. OnWatchDate : 사건을 담당하는 직원이 해당 사건을 관리하는 날짜
28. AvailableDtTS : 유닛이 해당 사건에 대해 대응할 수 있는 시간

In [2]:
from pyspark.sql.types import *

schema = StructType([
    StructField("CallNumber", IntegerType(), True),
    StructField("UnitID", StringType(), True),
    StructField("IncidentNumber", IntegerType(), True),
    StructField("CallType", StringType(), True),
    StructField("CallDate", StringType(), True),  
    StructField("WatchDate", StringType(), True), 
    StructField("CallFinalDisposition", StringType(), True),
    StructField("AvailableDtTm", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Zipcode", IntegerType(), True),
    StructField("Battalion", StringType(), True),
    StructField("StationArea", StringType(), True),
    StructField("Box", StringType(), True),
    StructField("OriginalPriority", StringType(), True),
    StructField("Priority", StringType(), True),
    StructField("FinalPriority", IntegerType(), True),
    StructField("ALSUnit", BooleanType(), True),
    StructField("CallTypeGroup", StringType(), True),
    StructField("NumAlarms", IntegerType(), True),
    StructField("UnitType", StringType(), True),
    StructField("UnitSequenceInCallDispatch", IntegerType(), True),
    StructField("FirePreventionDistrict", StringType(), True),
    StructField("SupervisorDistrict", StringType(), True),
    StructField("Neighborhood", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("RowID", StringType(), True),
    StructField("Delay", FloatType(), True)
])

In [3]:
df = spark.read.csv('./data/sf-fire-calls.csv', header=True, schema=schema)
df.show(5)

24/12/10 10:01:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

In [4]:
df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [6]:
# 열 이름 변경
from pyspark.sql import functions as F

df = df.withColumnRenamed('Delay', 'ResponseDelayedinMins')
df.select('ResponseDelayedinMins')\
    .where(F.col('ResponseDelayedinMins') > 5)\
    .show(5, False)

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|5.35                 |
|6.25                 |
|5.2                  |
|5.6                  |
|7.25                 |
+---------------------+
only showing top 5 rows



In [7]:
df.select('CallDate','WatchDate','AvailableDtTm').show(5, False)

+----------+----------+----------------------+
|CallDate  |WatchDate |AvailableDtTm         |
+----------+----------+----------------------+
|01/11/2002|01/10/2002|01/11/2002 01:51:44 AM|
|01/11/2002|01/10/2002|01/11/2002 03:01:18 AM|
|01/11/2002|01/10/2002|01/11/2002 02:39:50 AM|
|01/11/2002|01/10/2002|01/11/2002 04:16:46 AM|
|01/11/2002|01/10/2002|01/11/2002 06:01:58 AM|
+----------+----------+----------------------+
only showing top 5 rows



In [8]:
# 데이터 타입 변경
df = df.withColumn('IncidentDate', F.to_timestamp(F.col('CallDate'), 'MM/dd/yyyy'))\
    .drop('CallDate')\
    .withColumn('OnWatchDate', F.to_timestamp(F.col('WatchDate'), 'MM/dd/yyyy'))\
    .drop('WatchDate')\
    .withColumn('AvailableDtTS', F.to_timestamp(F.col('AvailableDtTm'),'MM/dd/yyyy hh:mm:ss a'))\
    .drop('AvailableDtTm')
df.select('IncidentDate','OnWatchDate','AvailableDtTS').show(5, False)

+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTS      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:44|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 03:01:18|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:39:50|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 04:16:46|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 06:01:58|
+-------------------+-------------------+-------------------+
only showing top 5 rows



## CallType

In [9]:
# 가장 흔한 CallType
df.select('CallType').where(F.col('CallType').isNotNull()).groupBy('CallType')\
    .count().orderBy('count', ascending=False).show(10, False)

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
+-------------------------------+------+
only showing top 10 rows



In [11]:
# CallType 종류수
df.select('CallType') \
    .where(F.col('CallType').isNotNull()) \
    .agg(F.countDistinct('CallType').alias('DistinctCallTypes')) \
    .select('DistinctCallTypes') \
    .show()

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



In [12]:
# CallType 종류
df.select('CallType')\
    .where(F.col('CallType').isNotNull())\
    .distinct().show(10, truncate=False)

+-----------------------------------+
|CallType                           |
+-----------------------------------+
|Elevator / Escalator Rescue        |
|Marine Fire                        |
|Aircraft Emergency                 |
|Confined Space / Structure Collapse|
|Administrative                     |
|Alarms                             |
|Odor (Strange / Unknown)           |
|Citizen Assist / Service Call      |
|HazMat                             |
|Watercraft in Distress             |
+-----------------------------------+
only showing top 10 rows



In [15]:
# 'Medical Incident'가 아닌 CallType

df.select('IncidentNumber','AvailableDtTS','CallType')\
    .where(F.col('CallType') != 'Medical Incident').show(5, truncate=False)

+--------------+-------------------+--------------+
|IncidentNumber|AvailableDtTS      |CallType      |
+--------------+-------------------+--------------+
|2003235       |2002-01-11 01:51:44|Structure Fire|
|2003250       |2002-01-11 04:16:46|Vehicle Fire  |
|2003259       |2002-01-11 06:01:58|Alarms        |
|2003279       |2002-01-11 08:03:26|Structure Fire|
|2003301       |2002-01-11 09:46:44|Alarms        |
+--------------+-------------------+--------------+
only showing top 5 rows



In [21]:
# CallType별 지연시간
df.filter(F.col('ResponseDelayedinMins').isNotNull())\
    .groupBy('CallType')\
    .agg(F.avg('ResponseDelayedinMins').alias('avg_ResponseDelayedinMins'))\
    .orderBy('avg_ResponseDelayedinMins', ascending=False)\
    .show(truncate=False)

+-----------------------------------+-------------------------+
|CallType                           |avg_ResponseDelayedinMins|
+-----------------------------------+-------------------------+
|Mutual Aid / Assist Outside Agency |38.41666640175713        |
|Assist Police                      |26.981903961726598       |
|Train / Rail Incident              |16.452046751976013       |
|Administrative                     |12.261111179987589       |
|HazMat                             |7.527016133069992        |
|Marine Fire                        |6.928571377481733        |
|Confined Space / Structure Collapse|6.915384567700899        |
|Watercraft in Distress             |6.886904835700989        |
|Suspicious Package                 |6.576666704813639        |
|High Angle Rescue                  |6.048958357423544        |
|Water Rescue                       |5.507748341362998        |
|Other                              |5.505155435636888        |
|Fuel Spill                         |5.4

## IncidentDate

In [12]:
# IncidentDate 컬럼의 연도를 추출하여 고유한 연도 추출
df.select(F.year('IncidentDate')).distinct()\
    .orderBy(F.year('IncidentDate')).show()

+------------------+
|year(IncidentDate)|
+------------------+
|              2000|
|              2001|
|              2002|
|              2003|
|              2004|
|              2005|
|              2006|
|              2007|
|              2008|
|              2009|
|              2010|
|              2011|
|              2012|
|              2013|
|              2014|
|              2015|
|              2016|
|              2017|
|              2018|
+------------------+



## ResponseDelayedinMins

In [15]:
# 경보횟수 합, 응답시간 평균, 모든 신고에 대한 최소/최장 응답시간
df.select(F.sum('NumAlarms'), F.avg('ResponseDelayedinMins'), F.min('ResponseDelayedinMins'),
          F.max('ResponseDelayedinMins')).show()

+--------------+--------------------------+--------------------------+--------------------------+
|sum(NumAlarms)|avg(ResponseDelayedinMins)|min(ResponseDelayedinMins)|max(ResponseDelayedinMins)|
+--------------+--------------------------+--------------------------+--------------------------+
|        176170|         3.892364154521585|               0.016666668|                   1844.55|
+--------------+--------------------------+--------------------------+--------------------------+



## City

In [25]:
# 도시별 호출빈도, 평균 응답시간
df.groupBy('City').agg(F.count('CallNumber').alias('CallFrequency'),
                       F.avg('ResponseDelayedinMins').alias('AvgResponse'))\
    .orderBy('CallFrequency', ascending=False).show(truncate=False)

+---------------+-------------+------------------+
|City           |CallFrequency|AvgResponse       |
+---------------+-------------+------------------+
|SF             |120072       |3.775843105805427 |
|San Francisco  |51739        |4.062894208634196 |
|SAN FRANCISCO  |1676         |4.594391408445301 |
|TI             |486          |4.974931401235086 |
|Presidio       |281          |4.991518377091112 |
|PR             |257          |4.769390379538912 |
|Treasure Isla  |228          |5.319956149709852 |
|null           |207          |14.03888882310602 |
|Yerba Buena    |41           |5.563008085983555 |
|DC             |41           |5.059349624122062 |
|SFO            |37           |4.251351350062603 |
|Fort Mason     |36           |4.457870363361305 |
|FM             |36           |4.030555579397413 |
|YB             |34           |10.709803798619438|
|HP             |31           |3.961290313351539 |
|Hunters Point  |27           |3.952469148017742 |
|PRESIDIO       |13           |

## StationArea

In [45]:
# null값 삭제
df = df.filter(df["StationArea"].isNotNull())

In [46]:
# 소방서별 지연 시간, 사건 수

df.groupBy('StationArea').agg(F.avg('ResponseDelayedinMins').alias('AvgResponse'),
                             F.count('IncidentNumber').alias('IncidentCount'))\
                        .show(truncate=False)

+-----------+------------------+-------------+
|StationArea|AvgResponse       |IncidentCount|
+-----------+------------------+-------------+
|07         |3.7866731589128566|8723         |
|51         |4.672758456990794 |777          |
|15         |3.7585408146960706|3173         |
|11         |3.69419441108856  |3755         |
|29         |3.92335887940679  |2610         |
|42         |4.321341404874807 |2731         |
|34         |4.0121785537644765|2061         |
|01         |3.8104291145308773|22434        |
|28         |4.099372758232992 |4464         |
|22         |3.9609293747011725|2629         |
|A2         |3.814985384630878 |684          |
|35         |4.40039249725103  |2845         |
|16         |3.7531507772942168|3597         |
|47         |4.0633333457840815|45           |
|43         |3.796693497280957 |4224         |
|31         |3.793202063818521 |3555         |
|18         |3.6552792877594   |2393         |
|27         |4.235519119217748 |244          |
|17         |

In [47]:
# 소방서별 CallType
df.groupBy("StationArea", "CallType").count().show(truncate=False)

+-----------+-----------------------------+-----+
|StationArea|CallType                     |count|
+-----------+-----------------------------+-----+
|03         |Elevator / Escalator Rescue  |72   |
|37         |Traffic Collision            |42   |
|A1         |Water Rescue                 |3    |
|39         |Citizen Assist / Service Call|58   |
|34         |Electrical Hazard            |15   |
|36         |Fuel Spill                   |8    |
|38         |Assist Police                |1    |
|42         |Odor (Strange / Unknown)     |5    |
|13         |HazMat                       |11   |
|02         |HazMat                       |1    |
|05         |Alarms                       |858  |
|15         |Outside Fire                 |21   |
|06         |Explosion                    |2    |
|48         |Elevator / Escalator Rescue  |1    |
|18         |Alarms                       |141  |
|02         |Odor (Strange / Unknown)     |9    |
|21         |Traffic Collision            |187  |


In [48]:
# 소방서와 CallType별 평균 지연시간
df.rollup('StationArea', 'CallType') \
    .agg(F.avg('ResponseDelayedinMins').alias('AvgResponseTime')) \
    .orderBy('StationArea', 'CallType') \
    .show(truncate=False)

+-----------+--------------------------------------------+------------------+
|StationArea|CallType                                    |AvgResponseTime   |
+-----------+--------------------------------------------+------------------+
|null       |null                                        |3.877523619094548 |
|01         |null                                        |3.8104291145308773|
|01         |Administrative                              |3.0               |
|01         |Alarms                                      |3.9778763818487826|
|01         |Assist Police                               |5.487500011920929 |
|01         |Citizen Assist / Service Call               |5.2691840169330435|
|01         |Confined Space / Structure Collapse         |3.0833332538604736|
|01         |Electrical Hazard                           |7.885714156287057 |
|01         |Elevator / Escalator Rescue                 |4.431124503353992 |
|01         |Explosion                                   |4.7952

## OriginalPriority, Priority, FinalPriority

In [33]:
# 호출 우선순위에 따른 지연시간
df.groupBy('OriginalPriority').agg(F.avg('ResponseDelayedinMins').alias('AvgResponse'))\
    .orderBy('OriginalPriority', ascending=True).show(truncate=False)

+----------------+------------------+
|OriginalPriority|AvgResponse       |
+----------------+------------------+
|null            |6.694558955397691 |
|1               |5.337625653748358 |
|2               |4.910639418509445 |
|3               |3.6404840559928235|
|A               |3.011516133027692 |
|B               |2.8238012383923174|
|C               |2.5783730173040005|
|E               |3.392433251869964 |
|I               |2.237349397626268 |
+----------------+------------------+



In [37]:
# null값 삭제
df = df.filter(df["OriginalPriority"].isNotNull())
df.groupBy('OriginalPriority').agg(F.avg('ResponseDelayedinMins').alias('AvgResponse'))\
    .orderBy('OriginalPriority', ascending=True).show(truncate=False)

+----------------+------------------+
|OriginalPriority|AvgResponse       |
+----------------+------------------+
|1               |5.337625653748358 |
|2               |4.910639418509445 |
|3               |3.6404840559928235|
|A               |3.011516133027692 |
|B               |2.8238012383923174|
|C               |2.5783730173040005|
|E               |3.392433251869964 |
|I               |2.237349397626268 |
+----------------+------------------+



In [30]:
# 사건 우선순위에 따른 지연 시간
df.groupBy('Priority').agg(F.avg('ResponseDelayedinMins').alias('AvgResponse'))\
    .orderBy('Priority', ascending=True).show(truncate=False)

+--------+------------------+
|Priority|AvgResponse       |
+--------+------------------+
|1       |5.162969258732872 |
|2       |4.611481444619451 |
|3       |3.6863685465231164|
|A       |4.201736083461179 |
|B       |4.903787863977028 |
|C       |4.610416620969772 |
|E       |3.5395174973000785|
|I       |3.2333333492279053|
+--------+------------------+



In [34]:
# 최종 우선순위에 따른 지연시간
df.groupBy('FinalPriority').agg(F.avg('ResponseDelayedinMins').alias('AvgResponse'))\
    .orderBy('FinalPriority', ascending=True).show(truncate=False)

+-------------+------------------+
|FinalPriority|AvgResponse       |
+-------------+------------------+
|2            |4.708378853760759 |
|3            |3.6782840658888674|
+-------------+------------------+

